In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
import time
import copy
from tqdm.notebook import tqdm

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


In [ ]:
# Data Loading and Preprocessing

# Define transformations
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
])

# Load CIFAR-10 dataset
trainset = CIFAR10(root='./data', train=True, download=True, transform=transform_train)
testset = CIFAR10(root='./data', train=False, download=True, transform=transform_test)

# Create data loaders
batch_size = 128
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

# CIFAR-10 classes
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


In [ ]:
# Visualize some training images
def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    npimg = img.numpy()
    plt.figure(figsize=(10, 4))
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.axis('off')
    plt.show()

# Get random training images
dataiter = iter(trainloader)
images, labels = next(dataiter)

# Show images
imshow(torchvision.utils.make_grid(images[:8]))
print(' '.join(f'{classes[labels[j]]:5s}' for j in range(8)))


In [ ]:
# Define the training and evaluation functions

def train_model(model, criterion, optimizer, scheduler=None, num_epochs=40):
    """
    Train the model and evaluate on test set
    """
    history = {
        'train_loss': [], 'train_acc': [],
        'test_loss': [], 'test_acc': []
    }
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)
        
        # Training phase
        model.train()
        running_loss = 0.0
        running_corrects = 0
        
        # Iterate over data
        for inputs, labels in tqdm(trainloader, desc="Training"):
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # Backward pass and optimize
            loss.backward()
            optimizer.step()
            
            # Statistics
            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            running_corrects += torch.sum(preds == labels.data)
        
        if scheduler is not None:
            scheduler.step()
            
        epoch_loss = running_loss / len(trainset)
        epoch_acc = running_corrects.double() / len(trainset)
        
        history['train_loss'].append(epoch_loss)
        history['train_acc'].append(epoch_acc.item())
        
        print(f'Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
        
        # Evaluation phase
        model.eval()
        running_loss = 0.0
        running_corrects = 0
        
        # No gradient computation for evaluation
        with torch.no_grad():
            for inputs, labels in tqdm(testloader, desc="Testing"):
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                # Forward pass
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                # Statistics
                running_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                running_corrects += torch.sum(preds == labels.data)
        
        epoch_loss = running_loss / len(testset)
        epoch_acc = running_corrects.double() / len(testset)
        
        history['test_loss'].append(epoch_loss)
        history['test_acc'].append(epoch_acc.item())
        
        print(f'Test Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
        
        # Deep copy the model if it's the best accuracy
        if epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())
        
        print()
    
    print(f'Best test accuracy: {best_acc:.4f}')
    
    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model, history


def plot_training_history(history, title):
    """
    Plot training and testing loss and accuracy
    """
    plt.figure(figsize=(12, 4))
    
    # Plot training and validation loss
    plt.subplot(1, 2, 1)
    plt.plot(history['train_loss'], label='Train')
    plt.plot(history['test_loss'], label='Test')
    plt.title(f'{title} - Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    # Plot training and validation accuracy
    plt.subplot(1, 2, 2)
    plt.plot(history['train_acc'], label='Train')
    plt.plot(history['test_acc'], label='Test')
    plt.title(f'{title} - Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.tight_layout()
    plt.show()


In [ ]:
## Part (a): Implementation of CNN Architectures

### 1. AlexNet Architecture
class AlexNet(nn.Module):
    def __init__(self, num_classes=10):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(64, 192, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 4 * 4, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x


In [ ]:
### 2. VGGNet Architecture (VGG16)
class VGG16(nn.Module):
    def __init__(self, num_classes=10):
        super(VGG16, self).__init__()
        # Block 1
        self.block1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        # Block 2
        self.block2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        # Block 3
        self.block3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        # Block 4
        self.block4 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        # Block 5
        self.block5 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        # Classifier
        self.classifier = nn.Sequential(
            nn.Linear(512, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, num_classes)
        )
        
    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x


In [ ]:
### 3. ResNet Architecture (ResNet18)

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(
            planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])


In [ ]:
### Train AlexNet
# Initialize model, loss function, and optimizer
alexnet = AlexNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(alexnet.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=40)

# Train the model
alexnet, alexnet_history = train_model(alexnet, criterion, optimizer, scheduler, num_epochs=40)

# Plot training history
plot_training_history(alexnet_history, 'AlexNet')


In [ ]:
### Train VGG16
# Initialize model, loss function, and optimizer
vgg16 = VGG16().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(vgg16.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=40)

# Train the model
vgg16, vgg16_history = train_model(vgg16, criterion, optimizer, scheduler, num_epochs=40)

# Plot training history
plot_training_history(vgg16_history, 'VGG16')


In [ ]:
### Train ResNet18
# Initialize model, loss function, and optimizer
resnet18 = ResNet18().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet18.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=40)

# Train the model
resnet18, resnet18_history = train_model(resnet18, criterion, optimizer, scheduler, num_epochs=40)

# Plot training history
plot_training_history(resnet18_history, 'ResNet18')


In [ ]:
## Part (b): Different Optimization Algorithms

### Train AlexNet with RMSProp
# Initialize model and loss function
alexnet_rmsprop = AlexNet().to(device)
criterion = nn.CrossEntropyLoss()

# RMSProp optimizer
optimizer_rmsprop = optim.RMSprop(alexnet_rmsprop.parameters(), lr=0.001, alpha=0.99, eps=1e-08, weight_decay=5e-4)

# Train the model
alexnet_rmsprop, alexnet_rmsprop_history = train_model(alexnet_rmsprop, criterion, optimizer_rmsprop, num_epochs=40)

# Plot training history
plot_training_history(alexnet_rmsprop_history, 'AlexNet with RMSProp')


In [ ]:
### Train AlexNet with Adam
# Initialize model and loss function
alexnet_adam = AlexNet().to(device)
criterion = nn.CrossEntropyLoss()

# Adam optimizer
optimizer_adam = optim.Adam(alexnet_adam.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=5e-4)

# Train the model
alexnet_adam, alexnet_adam_history = train_model(alexnet_adam, criterion, optimizer_adam, num_epochs=40)

# Plot training history
plot_training_history(alexnet_adam_history, 'AlexNet with Adam')


In [ ]:
### Compare Optimizers
# Plot comparison of different optimizers
plt.figure(figsize=(12, 8))

# Plot test accuracy
plt.subplot(2, 1, 1)
plt.plot(alexnet_history['test_acc'], label='SGD')
plt.plot(alexnet_rmsprop_history['test_acc'], label='RMSProp')
plt.plot(alexnet_adam_history['test_acc'], label='Adam')
plt.title('Test Accuracy Comparison')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot test loss
plt.subplot(2, 1, 2)
plt.plot(alexnet_history['test_loss'], label='SGD')
plt.plot(alexnet_rmsprop_history['test_loss'], label='RMSProp')
plt.plot(alexnet_adam_history['test_loss'], label='Adam')
plt.title('Test Loss Comparison')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
## Part (c): Improving Model Performance

### Method 1: Data Augmentation

# Define enhanced data augmentation
transform_train_augmented = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
])

# Load CIFAR-10 dataset with enhanced augmentation
trainset_augmented = CIFAR10(root='./data', train=True, download=True, transform=transform_train_augmented)
trainloader_augmented = DataLoader(trainset_augmented, batch_size=batch_size, shuffle=True, num_workers=2)

# Initialize ResNet18 model (our best model from part a)
resnet18_augmented = ResNet18().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet18_augmented.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=40)

# Modify the train_model function to use the augmented trainloader
def train_model_augmented(model, criterion, optimizer, trainloader, testloader, scheduler=None, num_epochs=40):
    """
    Train the model with custom data loaders and evaluate on test set
    """
    history = {
        'train_loss': [], 'train_acc': [],
        'test_loss': [], 'test_acc': []
    }
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)
        
        # Training phase
        model.train()
        running_loss = 0.0
        running_corrects = 0
        
        # Iterate over data
        for inputs, labels in tqdm(trainloader, desc="Training"):
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # Backward pass and optimize
            loss.backward()
            optimizer.step()
            
            # Statistics
            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            running_corrects += torch.sum(preds == labels.data)
        
        if scheduler is not None:
            scheduler.step()
            
        epoch_loss = running_loss / len(trainset)
        epoch_acc = running_corrects.double() / len(trainset)
        
        history['train_loss'].append(epoch_loss)
        history['train_acc'].append(epoch_acc.item())
        
        print(f'Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
        
        # Evaluation phase
        model.eval()
        running_loss = 0.0
        running_corrects = 0
        
        # No gradient computation for evaluation
        with torch.no_grad():
            for inputs, labels in tqdm(testloader, desc="Testing"):
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                # Forward pass
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                # Statistics
                running_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                running_corrects += torch.sum(preds == labels.data)
        
        epoch_loss = running_loss / len(testset)
        epoch_acc = running_corrects.double() / len(testset)
        
        history['test_loss'].append(epoch_loss)
        history['test_acc'].append(epoch_acc.item())
        
        print(f'Test Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
        
        # Deep copy the model if it's the best accuracy
        if epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())
        
        print()
    
    print(f'Best test accuracy: {best_acc:.4f}')
    
    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model, history

# Train the model with augmented data
resnet18_augmented, resnet18_augmented_history = train_model_augmented(
    resnet18_augmented, criterion, optimizer, trainloader_augmented, testloader, scheduler, num_epochs=40
)

# Plot training history
plot_training_history(resnet18_augmented_history, 'ResNet18 with Enhanced Data Augmentation')


In [ ]:
### Method 2: One Cycle Learning Rate Policy

# Initialize ResNet18 model
resnet18_onecycle = ResNet18().to(device)
criterion = nn.CrossEntropyLoss()

# Set up optimizer
optimizer = optim.SGD(resnet18_onecycle.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)

# One Cycle Learning Rate Scheduler
steps_per_epoch = len(trainloader)
scheduler = optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=0.1,
    steps_per_epoch=steps_per_epoch,
    epochs=40,
    pct_start=0.3,  # Percentage of training to increase the learning rate
    anneal_strategy='cos'  # Cosine annealing
)

# Modify train function to update scheduler every batch
def train_model_onecycle(model, criterion, optimizer, scheduler, num_epochs=40):
    """
    Train the model with OneCycleLR scheduler (updates every batch)
    """
    history = {
        'train_loss': [], 'train_acc': [],
        'test_loss': [], 'test_acc': [],
        'lr': []
    }
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)
        
        # Training phase
        model.train()
        running_loss = 0.0
        running_corrects = 0
        
        # Iterate over data
        for inputs, labels in tqdm(trainloader, desc="Training"):
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # Backward pass and optimize
            loss.backward()
            optimizer.step()
            
            # Update learning rate
            scheduler.step()
            
            # Statistics
            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            running_corrects += torch.sum(preds == labels.data)
        
        epoch_loss = running_loss / len(trainset)
        epoch_acc = running_corrects.double() / len(trainset)
        
        history['train_loss'].append(epoch_loss)
        history['train_acc'].append(epoch_acc.item())
        history['lr'].append(optimizer.param_groups[0]['lr'])
        
        print(f'Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f} LR: {optimizer.param_groups[0]["lr"]:.6f}')
        
        # Evaluation phase
        model.eval()
        running_loss = 0.0
        running_corrects = 0
        
        # No gradient computation for evaluation
        with torch.no_grad():
            for inputs, labels in tqdm(testloader, desc="Testing"):
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                # Forward pass
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                # Statistics
                running_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                running_corrects += torch.sum(preds == labels.data)
        
        epoch_loss = running_loss / len(testset)
        epoch_acc = running_corrects.double() / len(testset)
        
        history['test_loss'].append(epoch_loss)
        history['test_acc'].append(epoch_acc.item())
        
        print(f'Test Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
        
        # Deep copy the model if it's the best accuracy
        if epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())
        
        print()
    
    print(f'Best test accuracy: {best_acc:.4f}')
    
    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model, history

# Train the model with OneCycleLR
resnet18_onecycle, resnet18_onecycle_history = train_model_onecycle(
    resnet18_onecycle, criterion, optimizer, scheduler, num_epochs=40
)

# Plot training history
plot_training_history(resnet18_onecycle_history, 'ResNet18 with OneCycleLR')


In [ ]:
### Compare Improvement Methods
# Plot comparison of different improvement methods
plt.figure(figsize=(12, 8))

# Plot test accuracy
plt.subplot(2, 1, 1)
plt.plot(resnet18_history['test_acc'], label='ResNet18 (Baseline)')
plt.plot(resnet18_augmented_history['test_acc'], label='ResNet18 + Data Augmentation')
plt.plot(resnet18_onecycle_history['test_acc'], label='ResNet18 + OneCycleLR')
plt.title('Test Accuracy Comparison')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot test loss
plt.subplot(2, 1, 2)
plt.plot(resnet18_history['test_loss'], label='ResNet18 (Baseline)')
plt.plot(resnet18_augmented_history['test_loss'], label='ResNet18 + Data Augmentation')
plt.plot(resnet18_onecycle_history['test_loss'], label='ResNet18 + OneCycleLR')
plt.title('Test Loss Comparison')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

# Plot learning rate schedule for OneCycleLR
plt.figure(figsize=(10, 4))
plt.plot(resnet18_onecycle_history['lr'])
plt.title('OneCycleLR Learning Rate Schedule')
plt.xlabel('Epoch')
plt.ylabel('Learning Rate')
plt.show()
